In [29]:
3000*3000 / (256*512*128)

0.5364418029785156

In [2]:
using Revise, DeconvOptim, Napari, Images, FourierTools, Napari, FFTW, TestImages, Noise, CUDA, Zygote, LinearAlgebra
FFTW.set_num_threads(6)

In [3]:
CUDA.allowscalar(false)

In [4]:
img = Float32.(load("../../../../Documents/Deconvolution_Data/microtubules-challenge/data.tif"));
psf = ifftshift(Float32.(load("../../../../Documents/Deconvolution_Data/microtubules-challenge/psf.tif")));

img_c = CuArray(img)
psf_c = CuArray(psf);

┌ Warning: some versions of ImageMagick give spurious low-order bits for 32-bit TIFFs
└ @ ImageMagick /home/fxw/.julia/packages/ImageMagick/b8swT/src/ImageMagick.jl:100


In [36]:
reg = TV(num_dims=3, mode="forward")

#155 (generic function with 1 method)

In [5]:
reg_c = TV_cuda(num_dims=3)

#15 (generic function with 1 method)

In [51]:
loss = Poisson()

poisson_aux (generic function with 2 methods)

In [85]:
@time res_lr = richardson_lucy_iterative(1f-5 .+ img, psf, iterations=80, λ=0.002f0, regularizer=reg);

 20.955404 seconds (58.84 k allocations: 5.567 GiB, 1.92% gc time)


In [106]:
GC.gc(true)
CUDA.@time res_lr_c = richardson_lucy_iterative(1f-5 .+ img_c, psf_c, iterations=80, λ=0.002f0, regularizer=reg_c);

  3.490675 seconds (14.43 M CPU allocations: 229.052 MiB, 4.84% gc time) (3.33 k GPU allocations: 191.346 GiB, 20.15% gc time of which 75.06% spent allocating)


In [107]:
GC.gc(true)
@time res, o = deconvolution(img, psf, iterations=12, regularizer=reg, λ=0.002f0);
o

 15.282415 seconds (19.68 k allocations: 9.571 GiB, 20.83% gc time)


 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     4.936696e+06

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.10e-01 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.27e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.05e+01 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.63e-05 ≰ 0.0e+00
    |g(x)|                 = 2.86e-02 ≰ 1.0e-08

 * Work counters
    Seconds run:   14  (vs limit Inf)
    Iterations:    12
    f(x) calls:    15
    ∇f(x) calls:   13


In [8]:
GC.gc(true)
CUDA.@time res_c, o = deconvolution(img_c, psf_c, iterations=12, regularizer=reg_c, λ=0.002f0);
o

  1.286988 seconds (4.02 M CPU allocations: 65.351 MiB, 6.77% gc time) (1.16 k GPU allocations: 44.781 GiB, 25.32% gc time of which 72.43% spent allocating)


 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     4.936696e+06

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.10e-01 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.23e-02 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.00e+01 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.62e-05 ≰ 0.0e+00
    |g(x)|                 = 2.91e-02 ≰ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    12
    f(x) calls:    15
    ∇f(x) calls:   13


In [9]:
v = @view_image Array(res_c)
@add_image v res_lr
@add_image v res
@add_image v img

LoadError: UndefVarError: res_lr not defined

In [96]:
@show loss(FourierTools.conv(res_lr, psf), img) + 0.002f0 * reg(res_lr)
@show loss(FourierTools.conv(res, psf), img) + 0.002f0 * reg(res)

loss(FourierTools.conv(res_lr, psf), img) + 0.002f0 * reg(res_lr) = 210828.89f0
loss(FourierTools.conv(res, psf), img) + 0.002f0 * reg(res) = 210964.45f0


210964.45f0